# Network analysis of CRISPR essential genes in pancreatic cancer

------------

Author: Brin Rosenthal (sbrosenthal@ucsd.edu)

-----------

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import networkx as nx
import seaborn as sns

import community

import mygene
mg = mygene.MyGeneInfo()

# latex rendering of text in graphs
import matplotlib as mpl
mpl.rc('text', usetex = False)
mpl.rc('font', family = 'serif')

% matplotlib inline

import visJS2jupyter.visJS_module
import visJS2jupyter.visualizations


# Load the CRISPR data and RNAseq data

Update 12/07/17: Use intersection of RNAseq fdr significant genes and CRISPR significant genes as seeds

3DGvs3DV --> selects genes sensitive to drug   
3DVvsT21 --> selects genes essential for 'stemness'

In [2]:
# ---- Now updated file from Nikki ------

rna_df = pd.read_excel('../data/rnaseq_crispr_171207.xlsx',sheetname='RNA-seq',index_col='symbol')
rna_df.head()
# crispr_T21_df = pd.read_csv('../data/RS_MAGeCK_comparison/RS_T0_T21.gene_summary.txt',sep='\t',index_col='Unnamed: 0')
# crispr_T21_df.head()

# crispr_3DV_df = pd.read_csv('../data/RS_MAGeCK_comparison/RS_3DV.gene_summary.txt',sep='\t',index_col='Unnamed: 0')
# crispr_3DV_df.head(10)

,gene,description,NEG1,NEG2,NEG3,GFP1,GFP2,GFP3,NEG,GFP,log2 GFP/NEG,fc GFP/NEG,lfdr GFP-NEG,fdr GFP-NEG
symbol,,,,,,,,,,,,,,
Slc10a2,20494,"solute carrier family 10, member 2",3,4,5,198,202,218,4,206,5.436202,43.297196,0.000387,0.000387
G0s2,14373,G0/G1 switch gene 2,2933,2180,3846,23,27,31,2986,27,-6.731514,-106.264347,0.000621,0.000504
Cd177,68891,CD177 antigen,3455,3029,3022,52,79,79,3169,70,-5.513000,-45.664450,0.000688,0.000565
Il1r2,16178,"interleukin 1 receptor, type II",5689,7159,7494,125,130,77,6781,111,-5.959055,-62.209147,0.001087,0.000696
Gpr84,80910,G protein-coupled receptor 84,203,190,213,6,6,9,202,7,-4.795802,-27.776680,0.001274,0.000812


In [3]:
#crispr_3DV_df = pd.read_excel('../data/rnaseq_crispr_171207.xlsx',sheetname='Crispr 3DV',skiprows=1,
#                             index_col='Unnamed: 0')

# load new crispr data
crispr_3DV_df = pd.read_csv('../data/3DV_enhancers_p_fdr_180119.txt',sep='\t',index_col='Unnamed: 0')

crispr_3DV_df.head()

,alphaT_1,alphaT_2,alphaT_3,mean alphaT,permutation p,permutation FDR
mmu-mir-6979,-1.895770,-3.212108,-2.038180,-2.382019,4.718317e-08,0.0010
mmu-mir-669a-2,-2.579136,-2.589178,-0.826302,-1.998205,7.077475e-07,0.0075
Ssxb5,-2.012572,-1.778212,-1.890120,-1.893635,7.077475e-07,0.0050
mmu-mir-1249,-0.559810,-1.883050,-3.241975,-1.894945,2.076059e-06,0.0110
Defa26,-0.556956,-1.673242,-3.428127,-1.886108,2.406341e-06,0.0102


In [4]:
crispr_3DV_df.loc['Rorc']

alphaT_1          -0.888965
alphaT_2          -0.466446
alphaT_3          -1.190371
mean alphaT       -0.848594
permutation p      0.001485
permutation FDR    0.425392
Name: Rorc, dtype: float64

In [5]:
print(len(crispr_3DV_df[crispr_3DV_df['permutation FDR']<0.5]))

94


In [6]:
# intersection of crispr-significant and rna-expressed

rna_expressed = rna_df.index.tolist()

crispr_rna_expressed = crispr_3DV_df[crispr_3DV_df['permutation FDR']<0.5]
crispr_rna_expressed = crispr_rna_expressed.loc[rna_expressed].dropna()
crispr_rna_expressed = crispr_rna_expressed.sort_values('permutation FDR',ascending=True)

print(len(crispr_rna_expressed))
crispr_rna_expressed.head()
crispr_rna_expressed = crispr_rna_expressed.index.tolist() # cast to gene names

57


In [7]:
# intersection dataset filtered by FC

rna_FC2 = rna_df[rna_df['log2 GFP/NEG'].abs()>2]
crispr_rna_fc =  crispr_3DV_df[crispr_3DV_df['permutation FDR']<0.5] # filter by FDR
crispr_rna_fc = crispr_rna_fc.loc[rna_FC2.index.tolist()].dropna() # filter by rna FC
crispr_rna_fc = crispr_rna_fc.sort_values('permutation FDR',ascending=True)

print(len(crispr_rna_fc))
crispr_rna_fc
crispr_rna_fc = crispr_rna_fc.index.tolist() # cast to gene names

10


In [8]:
crispr_rna_fc

[u'Gm3776',
 u'BC100530',
 u'Cd300lh',
 u'Gm10334',
 u'Sox4',
 u'Klra18',
 u'Pard6b',
 u'Smo',
 u'Rorc',
 u'Ccdc120']

# Load Interactome (for now- mouse STRING)

Download from here:
https://string-db.org/cgi/download.pl?sessionId=Df4zRlbs4NEr&species_text=Mus+musculus

In [9]:
def load_STRING(datafile = '/Users/brin/Documents/CCBB_tickets_data/STRING/mouse/10090.protein.links.v10.5.txt',conf_thresh=700,species='mouse',
               cap_choice='lower'):
    '''
    Helper function to parse and load STRING network
    
    '''
    
    string_df = pd.read_csv(datafile,sep=' ')
    string_df = string_df.loc[string_df['combined_score']>conf_thresh]
                              
    # create the network
    G_str = nx.Graph()
    G_str.add_weighted_edges_from(zip(string_df['protein1'],string_df['protein2'],string_df['combined_score']))
    
    # convert from ensembleID to gene symbol
    string_genes_temp = [n[n.find('.')+1:] for n in G_str.nodes()]

    mg_temp = mg.querymany(string_genes_temp,scopes='ensemblprotein',fields='symbol',species=species)
    ensembl_list = [x['query'] for x in mg_temp]
    symbol_list = [x['symbol'] if 'symbol' in x.keys() else 'None' for x in mg_temp]
    ensembl_to_symbol = dict(zip(ensembl_list,symbol_list))
    ensembl_to_symbol = pd.Series(ensembl_to_symbol)
    
    # relabel nodes with symbols
    G_str = nx.relabel_nodes(G_str,dict(zip(G_str.nodes(),string_genes_temp)))
    G_str = nx.relabel_nodes(G_str,dict(ensembl_to_symbol[ensembl_to_symbol!='None']))  # only keep the proteins that have associated genes
    
    if cap_choice=='lower':
        genes = [g[0]+g[1:].lower() for g in G_str.nodes()]
        G_str = nx.relabel_nodes(G_str,dict(zip(G_str.nodes(),genes)))
    elif cap_choice=='upper':
        genes = [g.upper() for g in G_str.nodes()]
        G_str = nx.relabel_nodes(G_str,dict(zip(G_str.nodes(),genes)))
    
    return G_str

In [10]:
# HIGH CONFIDENCE STRING
#G_str = load_STRING(datafile='/Users/brin/Documents/CCBB_tickets_data/STRING/mouse/10090.protein.links.v10.5.txt',
#                                     conf_thresh=700,species='mouse',cap_choice='lower')

G_str = nx.read_gpickle('/Users/brin/Documents/CCBB_tickets_data/STRING/mouse/G_str.gpickle')

# MEDIUM CONFIDENCE STRING
# G_str = load_STRING(datafile='/Users/brin/Documents/CCBB_tickets_data/STRING/mouse/10090.protein.links.v10.5.txt',
#                                      conf_thresh=400,species='mouse',cap_choice='lower')

In [11]:
# remove UBC
if 'Ubc' in G_str.nodes():
    G_str.remove_node('Ubc')



In [12]:
'Lipt2' in G_str.nodes()

True

In [13]:
np.intersect1d(crispr_rna_fc,G_str.nodes())

array([u'Gm10334', u'Pard6b', u'Rorc', u'Smo', u'Sox4'], dtype='<U18')

In [14]:
len(G_str.edges())

410989

# Select which interactome to use

In [15]:
Gint = G_str 

if 'None' in Gint.nodes():
    Gint.remove_node('None')


In [16]:
# development versions of return_node_to_color and return_edge_to_color

def return_node_to_color(G,field_to_map='degree',cmap=mpl.cm.jet,alpha = 1.0, color_vals_transform = None,ceil_val=10,
                        color_max_frac = 1.0,color_min_frac = 0.0,vmin=None,vmax=None):
    

    '''
    Function to return a dictionary mapping nodes (keys) to colors (values), based on the selected field_to_map.
        - field_to_map must be a node attribute
        - cmap must be a valid matplotlib colormap
        - color_max_frac and color_min_frac allow user to set lower and upper ranges for colormap
    
    '''
    
    
    
    nodes_with_data = [(n[0],n[1][field_to_map]) for n in G.nodes(data=True)]
    
    
    if color_vals_transform == 'log':
        nodes,data = zip(*nodes_with_data)
        min_dn0 = np.nanmin([d for d in data if d>0])
        data = [np.log(np.max([d,min_dn0])) for d in data]  # set the zero d values to minimum non0 value
        data = [(d-np.nanmin(data)) for d in data] # shift so we don't have any negative values
        nodes_with_data = zip(nodes,data)
        
    elif color_vals_transform == 'sqrt':
        nodes,data = zip(*nodes_with_data)
        data = [np.sqrt(d) for d in data]
        nodes_with_data = zip(nodes,data)
        
    elif color_vals_transform == 'ceil':
        nodes,data = zip(*nodes_with_data)
        data = [min(d,ceil_val) for d in data]
        nodes_with_data = zip(nodes,data)
    else:
        nodes,data = zip(*nodes_with_data)
        
    # if vmin and vmax aren't set, set them to min and max of the data
    if vmin == None:
        vmin = np.nanmin(data)
    if vmax == None:
        vmax = np.nanmax(data)
        
    node_to_mapField = dict(nodes_with_data)
    
    color_to_mult = 256*(color_max_frac-color_min_frac)
    color_to_add = 256*color_min_frac
    print(color_to_mult)
    print(color_to_add)
    print(np.nanmax(list(node_to_mapField.values())))
    
    color_list = [np.multiply(cmap(int(float(node_to_mapField[d]-vmin)/(vmax-vmin)*color_to_mult+color_to_add)),256) 
                  if ~np.isnan(node_to_mapField[d])
                  else [np.nan]
                  for d in G.nodes()]
    
    color_list = [(int(c[0]),int(c[1]),int(c[2]),alpha) 
                  if ~np.isnan(c[0])
                  else (200,200,200,alpha)
                  for c in color_list]

    node_to_color = dict(zip(G.nodes(),['rgba'+str(c) for c in color_list]))
    
    return node_to_color

def return_edge_to_color(G,field_to_map='degree',cmap=mpl.cm.jet,alpha = 1.0, color_vals_transform = None,ceil_val=10,
                        vmin=None,vmax=None):
    
    
    '''
    Function to return a dictionary mapping edges (keys) to colors (values), based on the selected field_to_map.
        - field_to_map must be an edge attribute
        - cmap must be a valid matplotlib colormap
    
    '''
    
    edges_with_data = [(e[0],e[1],e[2][field_to_map]) for e in G.edges(data=True)]
    
    edges1,edges2,data = zip(*edges_with_data)
    
    
    
    
    if color_vals_transform == 'log':
        data = [np.log(d) for d in data]
        data = [(d-np.min(data)) for d in data] # shift so we don't have any negative values
        edges_with_data = zip(zip(edges1,edges2),data)
        
    elif color_vals_transform == 'sqrt':
        data = [np.sqrt(d) for d in data]
        edges_with_data = zip(zip(edges1,edges2),data)
        
    elif color_vals_transform == 'ceil':
        data = [max(d,ceil_val) for d in data]
        edges_with_data = zip(zip(edges1,edges2),data)
    else:
        
        edges_with_data = zip(zip(edges1,edges2),data)
        
    # if vmin and vmax aren't set, set them to min and max of the data
    if vmin == None:
        vmin = np.nanmin(data)
    if vmax == None:
        vmax = np.nanmax(data)
        
    edge_to_mapField = dict(edges_with_data)
    
    color_list = [np.multiply(cmap(int(float(edge_to_mapField[d]-vmin)/(vmax-vmin)*256)),256) for d in G.edges()]
    
    color_list = [(int(c[0]),int(c[1]),int(c[2]),alpha) for c in color_list]
    
    edge_to_color = dict(zip(G.edges(),['rgba'+str(c) for c in color_list]))
    
    return edge_to_color

def bias_position_by_partition(pos,partition,r=1.0,focal_partitions=None,scale_by_groupsize=False,
                              use_orig_angle=False):
    '''
    Bias the positions by partition membership, to group them together
    
    '''
    
    partition = pd.Series(partition)
    
    
    
    # if focal_partitions haven't been defined, use all of them
    if focal_partitions==None:
        focal_partitions = list(np.unique(partition))
    
    
    counter=-1
    for p in focal_partitions:
        counter+=1

        focal_nodes = partition[partition==p].index.tolist()
        
        # if use_orig_angle==True, move groups apart by their original angle
        if use_orig_angle:
            xlist = [pos[n][0] for n in focal_nodes]
            ylist = [pos[n][1] for n in focal_nodes]
            xmean = np.mean(xlist)
            ymean = np.mean(ylist)
            
            rho,phi = cart2pol(xmean,ymean)
        else:
            phi = float(counter)/(len(focal_partitions))*2*np.pi
        
        # if scale_by_groupsize==True, move smaller groups further towards periphery
        if scale_by_groupsize:
            rtemp=r*1.0/np.sqrt(len(focal_nodes))
        else:
            rtemp=r
        
        for n in focal_nodes:
            pos[n][0]+=pol2cart(rtemp,phi)[0]
            pos[n][1]+=pol2cart(rtemp,phi)[1]
            
    return pos

def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return(rho, phi)

def pol2cart(rho, phi):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return(x, y)

# Bottom-up approach: use network propagation to explore network neighborhood near genes of interest

- Start propagation from Roman's essential genes


In [17]:
# make a table of filtering steps, for MS
crispr_sig = crispr_3DV_df[crispr_3DV_df['permutation FDR']<0.5].index.tolist()
seed_filters_df = pd.DataFrame(np.zeros((len(crispr_sig),6)),index=crispr_sig)
seed_filters_df.columns=['CRISPR fdr','RNAseq logFC', 'CRISPR-essential','RNA-expressed','RNA-DE','in_STRING']
seed_filters_df['CRISPR fdr'].loc[crispr_sig]=crispr_3DV_df.loc[crispr_sig]['permutation FDR']
seed_filters_df['RNAseq logFC'].loc[crispr_sig]=rna_df.loc[crispr_sig]['log2 GFP/NEG']
seed_filters_df['CRISPR-essential'].loc[crispr_sig] = True

seed_filters_df['RNA-expressed'].loc[crispr_sig] = False
seed_filters_df['RNA-expressed'].loc[crispr_rna_expressed] = True

seed_filters_df['RNA-DE'].loc[crispr_sig] = False
seed_filters_df['RNA-DE'].loc[crispr_rna_fc] = True

seed_filters_df['in_STRING'].loc[crispr_sig]=False
crispr_sig_string = list(np.intersect1d(crispr_sig,G_str.nodes()))
seed_filters_df['in_STRING'].loc[crispr_sig_string] = True

#seed_filters_df.to_csv('network_prop_seed_filters.csv')

seed_filters_df.head(15)

/Users/brin/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,CRISPR fdr,RNAseq logFC,CRISPR-essential,RNA-expressed,RNA-DE,in_STRING
mmu-mir-6979,0.001000,NaN,True,False,False,False
mmu-mir-669a-2,0.007500,NaN,True,False,False,False
Ssxb5,0.005000,NaN,True,False,False,False
mmu-mir-1249,0.011000,NaN,True,False,False,False
Defa26,0.010200,NaN,True,False,False,False
Klra1,0.009000,NaN,True,False,False,False
Higd1c,0.014571,NaN,True,False,False,False
Gm3776,0.013375,2.363606,True,True,True,False
H2-T9,0.013444,-0.744901,True,True,False,False
E130309D02Rik,0.036200,-0.244569,True,True,False,False


In [18]:
crispr_rna_expressed


['Gm3776',
 'H2-T9',
 'E130309D02Rik',
 'Rpl3',
 'Rpia',
 'Psmb6',
 'Fam63b',
 'Car13',
 'Rps16',
 'Thoc2',
 'Lss',
 'BC100530',
 'Chek1',
 'Lipt2',
 'Mau2',
 'Zfp523',
 'Fastkd5',
 'Ucp2',
 'Cars2',
 'Prpf8',
 'C030039L03Rik',
 'Hnrnpk',
 'Idi1',
 'Cd300lh',
 'Dolk',
 'Polg',
 'Shfm1',
 'Spry1',
 'Sars2',
 'Foxo1',
 'Gm10334',
 'Sox4',
 'Klra18',
 'Pard6b',
 '2310036O22Rik',
 'Mrps24',
 'Senp6',
 'Zfp598',
 'Ptprm',
 'Slc25a36',
 'Sf1',
 'Kcmf1',
 'Fam32a',
 'Scin',
 'Srm',
 'Aaas',
 'Smo',
 'Dmap1',
 'Osgep',
 'Rorc',
 'Qtrtd1',
 'Ccdc120',
 'Mrps15',
 'Exoc3',
 'N6amt1',
 'Wdr1',
 'Wdr25']

In [19]:
# seed genes for network propagation

#seed_genes = list(np.intersect1d(crispr_rna_expressed,Gint.nodes()))

seed_genes = list(np.intersect1d(crispr_rna_fc,Gint.nodes()))
print(len(seed_genes))

5


In [20]:
# calculate the normalized adjacency matrix
Wprime = visJS2jupyter.visualizations.normalized_adj_matrix(Gint)

In [21]:
# run the network propagation simulation
Fnew = visJS2jupyter.visualizations.network_propagation(Gint,Wprime,seed_genes)

In [22]:
Fnew.sort_values(ascending=False,inplace=True)
Fnew.head(10)

Sox4       0.102689
Smo        0.101974
Rorc       0.101412
Gm10334    0.101252
Pard6b     0.101218
Ctnnb1     0.009732
Sox9       0.008959
Sox17      0.008796
Sry        0.008461
Sox7       0.008318
dtype: float64

In [23]:
Fnew_rank = pd.DataFrame(Fnew)
Fnew_rank['rank']=range(len(Fnew_rank))
Fnew_rank.loc[['Celsr1','Celsr2','Pear1','Megf11']]

,0,rank
Celsr1,0.000005,5001.0
Celsr2,0.000003,6056.0
Pear1,NaN,NaN
Megf11,NaN,NaN


In [24]:
# select the top 500 genes proximal to the seed set

hot_genes = Fnew.head(500).index.tolist()

# ---------- Just hot genes --------------------
G_hot = nx.subgraph(Gint,hot_genes)

G_hot = max(nx.connected_component_subgraphs(G_hot), key=len) # take the Largest connected component
len(G_hot.nodes())

500

In [25]:
'Wnt1' in G_hot.nodes()

True

In [26]:
# plot the hot subnetwork
# add pval and fc as node attributes
#nx.set_node_attributes(G_sub,'pval',dict(crispr_df.loc[G_sub.nodes()]['Pval']))
nx.set_node_attributes(G_hot,'3DV_FDR',
                       dict(crispr_3DV_df.loc[G_hot.nodes()]['permutation FDR']))
#nx.set_node_attributes(G_hot,'Statistic',
#                       dict(crispr_3DV_df.loc[G_hot.nodes()]['s']))


# add rna fc as node attributes
nx.set_node_attributes(G_hot,'rna_logFC',dict(rna_df.loc[G_hot.nodes()]['log2 GFP/NEG']))
nx.set_node_attributes(G_hot,'rna_lfdr',dict(rna_df.loc[G_hot.nodes()]['lfdr GFP-NEG']))

# cluster the network
partition = community.best_partition(G_hot)
nx.set_node_attributes(G_hot,'partition',partition)
partition= pd.Series(partition)

In [27]:
G_hot.nodes(data=True)[0]

(u'Pdlim5',
 {'3DV_FDR': 1.00943092154236,
  'partition': 0,
  'rna_lfdr': 0.9911060722,
  'rna_logFC': 0.4515499638})

In [28]:
from networkx.drawing.nx_agraph import graphviz_layout

In [29]:
#pos = nx.spring_layout(G_hot,weight=None)
pos = graphviz_layout(G_hot,prog='neato')
for n in pos.keys():
    pos[n]=np.array([pos[n][0]/200.0,pos[n][1]/200.0])

#pos = bias_position_by_partition(pos,partition,r=1.9,use_orig_angle=False,focal_partitions = [0,1,2,6,3,8,4,5,7])
#pos = bias_position_by_partition(pos,partition,r=1.9,use_orig_angle=False,focal_partitions = [0,1,2,4,3,5])
#pos = bias_position_by_partition(pos,partition,r=1.7,use_orig_angle=False, focal_partitions = [4,1,2,5,6,0,3])
pos = bias_position_by_partition(pos,partition,r=1.7,use_orig_angle=True)

In [30]:
len(G_hot.edges())

8335

In [46]:
# draw the network and save for export to cytoscape



nodes = G_hot.nodes()
nodes_data = G_hot.nodes(data=True)
edges = G_hot.edges()
edges_data = G_hot.edges(data=True)
numnodes = len(nodes)
numedges = len(edges)


node_to_title = {}
node_to_size = {}
node_to_HL = {}
node_to_shape={}
node_to_label={}
for node in nodes_data:
    node_to_title[node[0]]=node[0]+': crispr 3DV FDR = '+str(node[1]['3DV_FDR'])
    node_to_title[node[0]]+='<br/>group = ' + str(partition.loc[node[0]])
    node_to_title[node[0]]+='<br/>rna log2FC = ' + str(node[1]['rna_logFC'])
    node_to_title[node[0]]+='<br/>rna lfdr = ' + str(node[1]['rna_lfdr'])
    
    # only label seed nodes or significant nodes
    if node[0] in crispr_3DV_df.index:
        if (node[0] in seed_genes) or (np.abs(node[1]['rna_logFC'])>3.0): 
            node_to_label[node[0]]=str(node[0])
        else:
            node_to_label[node[0]]=''
    else:
        node_to_label[node[0]]=''

    #node_to_size[node[0]]=-np.log(node[1]['lfdr'])-1.2 # subtract log(0.3)
    #node_to_size[node[0]]=np.abs(node[1]['Statistic'])-5 # map size to statistic
    #node_to_size[node[0]]=np.abs(node[1]['rna_logFC'])
    #node_to_size[node[0]]=-np.log(node[1]['rna_lfdr'])*3+2
    node_to_size[node[0]]=-np.log(node[1]['3DV_FDR'])*5+.1
    
    if node[0] in seed_genes:
        node_to_HL[node[0]]=1
    else:
        node_to_HL[node[0]]=0
        
#     if node[0] in seeds_crispr:
#         node_to_shape[node[0]]='triangle'
#     elif node[0] in seeds_rna:
#         node_to_shape[node[0]]='square'
    if node[0] in seed_genes:
        node_to_shape[node[0]]='triangle'
        node_to_size[node[0]]=25
    else:
        node_to_shape[node[0]]='dot'
        node_to_size[node[0]]=5

#node_to_color = return_node_to_color(G_hot,'partition',mpl.cm.Set1,color_max_frac=1)
#node_to_color = return_node_to_color(G_hot,'logFC_avg',mpl.cm.Blues_r,vmax=0,vmin=-3)
node_to_color = return_node_to_color(G_hot,'rna_logFC',mpl.cm.coolwarm,vmax=5,vmin=-5)
#node_to_color = return_node_to_color(G_hot,'Statistic',mpl.cm.autumn_r,vmax=5,vmin=0)

nodes_rna_logFC = nx.get_node_attributes(G_hot,'rna_logFC')
nodes_rna_lfdr = nx.get_node_attributes(G_hot,'rna_lfdr')
nodes_3DV_FDR = nx.get_node_attributes(G_hot,'3DV_FDR')
nodes_partition = nx.get_node_attributes(G_hot,'partition')
nodes_dict = [{"id":str(n),"color":node_to_color[n],
               "node_size":node_to_size[n],
               "border_width":node_to_HL[n],
               "node_shape":node_to_shape[n],
               "node_label":node_to_label[n],
                "rna_logFC":[nodes_rna_logFC[n] if not np.isnan(nodes_rna_logFC[n]) else 0.0][0],
               "rna_lfdr":[nodes_rna_lfdr[n] if not np.isnan(nodes_rna_lfdr[n]) else 0.0][0],
               "3DV_FDR":[nodes_3DV_FDR[n] if not np.isnan(nodes_3DV_FDR[n]) else 0.0][0],
               "partition":nodes_partition[n],
              "x":pos[n][0]*1000,
              "y":pos[n][1]*1000} for n in nodes
              ] # NOTE rna_logFC/lfdr = 0 means not expressed!

#                "title":node_to_title[n],

node_map = dict(zip(nodes,range(len(nodes))))  # map to indices for source/target in edges


# make the within-group edges darker
edge_to_color={}
for e in edges:
    p1 = partition[e[0]]
    p2 = partition[e[1]]
    if p1==p2:
        edge_to_color[e]="rgba(200, 250, 200,.3)"
    else:
        edge_to_color[e]="rgba(181, 227, 217,.2)"
#edge_to_color = return_edge_to_color(G_hot,field_to_map='weight',cmap=mpl.cm.BuGn,vmin=200,alpha=.2)
    
# only draw the strongest edges because we can't render all of them
edges_dict = [{"source":node_map[edges[i][0]], "target":node_map[edges[i][1]], 
              "color":edge_to_color[edges[i]],"title":'test'} for i in range(len(edges)) if edges_data[i][2]['weight']>700]

# set some network-wide styles
visJS2jupyter.visJS_module.visjs_network(nodes_dict,edges_dict,
                          node_size_multiplier=10,
                          node_size_field='node_size',
                          node_size_transform = 'Math.sqrt',
                          node_color_highlight_border='red',
                          node_color_highlight_background='#D3918B',
                          node_color_hover_border='blue',
                          node_color_border='white',
                          node_color_hover_background='#8BADD3',
                          node_label_field='node_label',
                          node_font_size=120,
                          node_font_color='rgba(240,240,240,1.0)',
                          physics_enabled=False,
                          edge_color_highlight='#8A324E',
                          edge_color_hover='#8BADD3',
                          edge_width=3, 
                          edge_smooth_enabled=False,
                          edge_smooth_type='cubicBezier', 
                          edge_smooth_roundness=0.5,
                          scaling_factor=1,
                          node_scaling_label_draw_threshold=0,
                          graph_id=1,
                          graph_title='',export_network=False,export_file='crispr_p5_rnaFC_medium_string.json')



256.0
0.0
5.262947675
500
500


In [48]:
# save hot subnetwork genes
print(len(seed_genes))
pd.Series(G_hot.nodes()).to_csv('G_hot.txt',index=False)

8


In [56]:
# print out genes in individual clusters
for n in partition[partition==4].index.tolist():
    print(n)

Aars
Abcc8
Aff1
Ambp
Apc
Avpi1
Chd4
Comp
Ctse
Ddx5
ENSMUSP00000127885
Eed
Eml1
Eno2
Epor
Ereg
Etv5
Fmnl2
Fth1
Gan
Gm128
Gpr182
Grap
Hopx
Ide
Ighg2c
Lat2
Mbnl1
Mst1r
Naa50
Nagpa
Ndp
Nin
Nrip3
Nts
Omp
Opn3
Ptprf
Ran
Rasl2-9
Rit2
Rorc
Sh3gl1
Slc5a5
Slc6a3
Son
Tap2
Tcf23
Trim37
Ubr5
Ust
Zbtb32
